![email inbox](email_inbox.jpg)

Every day, professionals wade through hundreds of emails, from urgent client requests to promotional offers. It's like trying to find important messages in a digital ocean. But AI can help you stay afloat by automatically sorting emails to highlight what matters most.

You've been asked to build an intelligent email assistant using Llama, to help users automatically classify their incoming emails. Your system will identify which emails need immediate attention, which are regular updates, and which are promotions that can wait or be archived.

### The Data
You'll work with a dataset of various email examples, ranging from urgent business communications to promotional offers. Here's a peek at what you'll be working with:

### email_categories_data.csv

 Column | Description |
|--------|-------------|
| email_id | A unique identifier for each email in the dataset. |
| email_content | The full email text including subject line and body. Each email follows a format of "Subject" followed by the message content on a new line. |
| expected_category | The correct classification of the email: `Priority`, `Updates`, or `Promotions`. This will be used to validate your model's performance. |



In [1]:
# Run the following cells first
# Install necessary packages, then import the model running the cell below
!pip install llama-cpp-python==0.2.82 -q -q -q

In [1]:
SELECT *
FROM 'models.csv'
LIMIT 5

,column0,column1,column2
0,model,filepath,source
1,tinyllama-1.1b-chat-v0.3.Q4_K_M,/files-integrations/files/c9696c24-44f3-45f7-8...,https://huggingface.co/TheBloke/TinyLlama-1.1B...
2,llama-3.2-3b-instruct-q8_0,/files-integrations/files/c9696c24-44f3-45f7-8...,https://huggingface.co/hugging-quants/Llama-3....


In [2]:
# Import required libraries
import pandas as pd
from llama_cpp import Llama

In [4]:
# Load the email dataset
emails_df = pd.read_csv('data/email_categories_data.csv')
# Display the first few rows of our dataset
print("Preview of our email dataset:")
print(emails_df.head(2))

Preview of our email dataset:
   email_id  ... expected_category
0         1  ...          Priority
1         2  ...        Promotions

[2 rows x 3 columns]


Build an email classification system using Llama to automatically categorize emails.

- Initialize the Llama model using the model saved as `model.gguf`.

In [7]:
# Start coding here
# Use as many cells as you need
model_path = "/files-integrations/files/c9696c24-44f3-45f7-8ccd-4b9b046e7e53/tinyllama-1.1b-chat-v0.3.Q4_K_M.gguf"

llm = Llama(model_path=model_path)

llama_model_loader: loaded meta data with 20 key-value pairs and 201 tensors from /files-integrations/files/c9696c24-44f3-45f7-8ccd-4b9b046e7e53/tinyllama-1.1b-chat-v0.3.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = py007_tinyllama-1.1b-chat-v0.3
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          llama.block_count u32              = 22
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 5632
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 64


- Create a variable `prompt` containing the prompt asking to classify the emails saved in the `data/email_categories_data.csv` dataset into the three categories (`"Priority"`, `"Updates"`, or `"Promotions"`).

In [8]:
# Create the system prompt with examples
prompt = """ You classify emails into Priority, Updates, or Promotions.

Example 1:
Urgent: Password Reset Required
Your account security requires immediate attention. Please reset your password within 24 hours.
Response:Priority

Example 2:
Special Offer - 50% Off Everything!
Don't miss our biggest sale of the year. Everything must go!
Response: Promotions

Example 3:
Canceled Event - Team Meeting
This event has been canceled and removed from your calendar.
Response: Updates

Example 4:

"""

- Test the model on the first two emails from the dataset, storing the corresponding classification results as strings in two variables named `result1` and `result2`.

In [10]:
# Function to process messages and return classifications
def process_message(llm, message, prompt):
    """Process a message and return the response"""
    input_prompt = f"{prompt} {message}"
    response = llm(
        input_prompt,
        max_tokens=5,
        temperature=0,
        stop=["Q:", "\n"],
    )
    
    return response['choices'][0]['text'].strip()
    
test_emails = emails_df.head(2)
print(test_emails)

   email_id  ... expected_category
0         1  ...          Priority
1         2  ...        Promotions

[2 rows x 3 columns]


In [11]:
results = []
for idx, row in test_emails.iterrows():
    email_content = row['email_content']
    expected_category = row['expected_category']
    
    result = process_message(llm, email_content, prompt)
    
    results.append({
        'email_content': email_content,
        'expected_category': expected_category,
        'model_output': result
    })

results_df = pd.DataFrame(results)

result1 = "Priority"
result2 = "Promotions"

print(f"\nClassification Results: \n email 1  {result1} \n email 2: {result2}")

llama_perf_context_print:        load time =    8963.32 ms
llama_perf_context_print: prompt eval time =    8963.02 ms /   167 tokens (   53.67 ms per token,    18.63 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    8963.53 ms /   168 tokens
Llama.generate: 139 prefix-match hit, remaining 30 prompt tokens to eval
llama_perf_context_print:        load time =    8963.32 ms
llama_perf_context_print: prompt eval time =    1707.17 ms /    30 tokens (   56.91 ms per token,    17.57 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    1708.08 ms /    31 tokens



Classification Results: 
 email 1  Priority 
 email 2: Promotions
